## Project: Rental Apartment Analysis Tel Aviv - Jaffa- Machine Learning – Part 3
### Presented by:  
### Amichay Nager -316225986 
### Tair Mimon-322240615
### github: https://github.com/amichay1234/project_part2-/blob/main/Machine%20Learning%20%E2%80%93%20Part%202%20Final.ipynb

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import cross_val_score, KFold,LeaveOneOut, LeavePOut, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error ,mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt
import seaborn as sns
import random
import pickle
import requests
import time
import getpass
import googlemaps
import movecolumn as mc

In [2]:
# Read the file
file_name = r'https://raw.githubusercontent.com/amichay1234/project_part2-/refs/heads/main/train.csv'
dataset = pd.read_csv(file_name)

# New column order
new_order = [
    "property_type", "neighborhood", "address", "room_num", "floor", "area", "garden_area", "days_to_enter", "num_of_payments",
    "monthly_arnona", "building_tax", "total_floors", "description", "has_parking", "has_storage", "elevator", "ac", "handicap", "has_bars",
    "has_safe_room", "has_balcony", "is_furnished", "is_renovated", "num_of_images", "distance_from_center", "price"]

# Change the order of the columns accordingly.
dataset = dataset[new_order]

# Displaying the first 5 lines
dataset.head(5)


,property_type,neighborhood,address,room_num,floor,area,garden_area,days_to_enter,num_of_payments,monthly_arnona,...,ac,handicap,has_bars,has_safe_room,has_balcony,is_furnished,is_renovated,num_of_images,distance_from_center,price
0,דירה,הצפון הישן החלק המרכזי,"מהר""ל 25",3.0,2,71,NaN,0.0,12.0,467.0,...,1,0,0,1,1,0,0,6.0,1005.0,10150.0
1,דירה,הצפון הישן החלק המרכזי,ארלוזורוב 35,3.0,1,70,NaN,0.0,12.0,240.0,...,1,0,1,0,1,0,0,3.0,253.0,6600.0
2,דירה,הצפון הישן החלק המרכזי,וורמיזה 5,2.5,1,65,NaN,NaN,12.0,400.0,...,1,1,0,0,1,0,1,8.0,740.0,9000.0
3,דירה,הצפון הישן החלק המרכזי,עמנואל הרומי 30,2.0,3,40,NaN,0.0,12.0,100.0,...,0,0,0,0,0,0,0,2.0,1206.0,5800.0
4,דירה,הצפון הישן החלק המרכזי,ארלוזורוב 50,3.0,1,70,NaN,0.0,11.0,250.0,...,1,0,1,0,0,0,1,5.0,255.0,7700.0


In [3]:
num_rows = dataset.shape[1]
print(f"Number of rows: {num_rows}")

num_columns = dataset.shape[0]
print(f"Number of columns: {num_columns}")

Number of rows: 26
Number of columns: 788


In [4]:
my_data_test=dataset.copy()

In [5]:
def prepare_data(dataset):

       
    
    
    
    
    
    if 'price' in dataset.columns:
        before = len(dataset)
        dataset.dropna(subset=['price'], inplace=True)
        print(f"❌ הוסרו {before - len(dataset)} שורות ללא price")

        # שלב 2 – שמירה על טווח מחירים סביר
        before = len(dataset)
        dataset = dataset[(dataset['price'] >= 1500) & (dataset['price'] <= 18500)]
        print(f"❌ הוסרו {before - len(dataset)} שורות עם מחיר מחוץ לטווח 1500–18500")
    else:
        print("⚠️ עמודת price לא קיימת – דילוג על שלבי סינון לפי מחיר")

    # שלב 3 – המרה של תווים חסרי מידע ל־NaN
    dataset.replace({'': np.nan, ' ': np.nan}, inplace=True)
    
    # שלב 4 – הסרת שורות בלי neighborhood ו־address גם יחד
    before = len(dataset)
    dataset = dataset[~(dataset["neighborhood"].isna() & dataset["address"].isna())].reset_index(drop=True)
    print(f"❌ הוסרו {before - len(dataset)} שורות שבהן גם neighborhood וגם address חסרים")

    # שלב 5 – הסרת שורות עם פחות מ־5 ערכים (שאינם 0 או NaN)
    before = len(dataset)
    dataset = dataset[dataset.apply(lambda row: (row != 0).sum() > 4, axis=1)]
    print(f"❌ הוסרו {before - len(dataset)} שורות עם 4 ערכים או פחות (לא 0 ולא NaN)")
    # סיכום כולל
   # print(f"\n📊 סך הכול נשארו {len(dataset)} שורות מתוך {start_rows} המקוריות.")

#################################################################################################################################

    # 🔹 שמירה על כמות שורות לפני עיבוד
    start_rows = len(dataset)
    
    # 🔹 שלב 1: סיווג לפי property_type
    def classify_property_type(val): 
        if not isinstance(val, str):
            return np.nan
        val_lower = val.lower()
        if 'סאבלט' in val_lower:
            return 'סאבלט'
        elif 'פרטי' in val_lower or 'קוטג' in val_lower or 'דו משפחתי' in val_lower:
            return 'פרטי'
        elif 'סטודיו' in val_lower or 'לופט' in val_lower:
            return 'סטודיו/לופט'
        elif 'גג' in val_lower or 'פנטהאוז' in val_lower:
            return 'גג/פנטהאוז'
        elif 'דופלקס' in val_lower:
            return 'דופלקס'
        elif 'דירת גן' in val_lower:
            return 'דירת גן'
        elif 'דירה' in val_lower or 'דירת' in val_lower:
            return 'דירה'
        else:
            return np.nan
    
    # 🔹 שלב 2: ניחוש לפי description
    def guess_type_from_description(desc):
        if pd.isna(desc):
            return np.nan
        desc = desc.lower()
        non_apartment_keywords = [
            r'משרד', r'משרדים', r'קליניקה', r'גלריה', r'סטודיו לאמן',
            r'חלל עבודה', r'תלת פאזי', r'מחסן', r'סדנה', r'בית מלאכה', r'מסחרי'
        ]
        for pattern in non_apartment_keywords:
            if re.search(rf'\b{pattern}\b', desc):
                return np.nan
        patterns = [
            (r'\bיחידת דיור\b', 'יחידת דיור'),
            (r'\bסאבלט\b', 'סאבלט'),
            (r'\b(פרטי|קוטג|דו משפחתי)\b', 'פרטי'),
            (r'\b(סטודיו|לופט)\b', 'סטודיו/לופט'),
            (r'\b(גג|פנטהאוז)\b', 'גג/פנטהאוז'),
            (r'\bדופלקס\b', 'דופלקס'),
            (r'\bדירת גן\b', 'דירת גן'),
            (r'\b(דירה|דירת)\b', 'דירה'),
            (r'\bחדר\b', 'דירה'),
            (r'\bבית\b', 'דירה'),
        ]
        for pattern, value in patterns:
            if re.search(pattern, desc):
                return value
        return np.nan
    
    # 🔹 החלה על הדאטה
    dataset['property_type_clean'] = dataset['property_type'].apply(classify_property_type)
    
    # 🔹 שלב השלמה לפי תיאור
    missing_mask = dataset['property_type_clean'].isna()
    dataset.loc[missing_mask, 'property_type_clean'] = dataset.loc[missing_mask, 'description'].apply(guess_type_from_description)
    
    # 🔹 מילוי "לא ידוע"
    dataset['property_type_clean'] = dataset['property_type_clean'].fillna('לא ידוע')
    
    # 🔹 סינון שורות עם "קורות חיים"
    before = len(dataset)
    dataset = dataset[~dataset['description'].str.contains('קורות חיים', case=False, na=False)]
    print(f"❌ הוסרו {before - len(dataset)} שורות עם התייחסות ל'קורות חיים'")
    
    # 🔹 הסרת שורות עם 'לא ידוע'
    before = len(dataset)
    dataset = dataset[dataset['property_type_clean'] != 'לא ידוע']
    print(f"❌ הוסרו {before - len(dataset)} שורות שבהן סוג הנכס לא זוהה (לא ידוע)")
    
    # 🔹 סיכום
    print(f"\n📊 סך הכול נשארו {len(dataset)} שורות מתוך {start_rows} המקוריות.")

####################################################################################################################################
    def split_address(address):
        if not isinstance(address, str):
            return pd.Series([None, None])
        match = re.search(r'(\D+)\s*(\d+)?', address)
        if match:
            street = match.group(1).strip()
            number = match.group(2) if match.group(2) else None
            return pd.Series([street, number])
        return pd.Series([None, None])

    dataset[['street', 'house_number']] = dataset['address'].apply(split_address)
    dataset.drop('house_number', axis=1, inplace=True)

    conditions = (dataset['floor'].isna()) & (dataset['property_type'].isin(['פרטי']))
    dataset.loc[conditions, 'floor'] = 0
    dataset['street'] = dataset['street'].fillna("Unknown")
######################################################################################################################
    def extract_floor_clean(val):
        if pd.isna(val):
            return np.nan
        val = str(val).strip()
        if val == 'קרקע מתוך קרקע':
            return 0
        if 'קרקע' in val:
            return 0
        if 'מרתף' in val:
            return -1
        match = re.findall(r'\d+', val)
        if match:
            try:
                num = int(match[0])
                if 0 <= num <= 100:
                    return num
            except:
                return np.nan
        return np.nan

    dataset['floor_clean'] = dataset.apply(lambda row: extract_floor_clean(row['floor']), axis=1)
    dataset['floor_clean'] = pd.to_numeric(dataset['floor_clean'], errors='coerce')
    condition = (dataset['total_floors'] == 0) & (dataset['floor_clean'].isin([0, 1]))
    dataset.loc[condition, 'total_floors'] = 1

    def extract_floor_strict(val, total_floors):
        if pd.isnull(val):
            return np.nan
        val = str(val).strip()
        if val == '0' or 'קרקע' in val:
            return 1
        match = re.search(r'\d+', val)
        if not match:
            return np.nan
        num_str = match.group()
        try:
            num = int(num_str)
        except:
            return np.nan
        if 0 < num <= 100:
            return num
        if not pd.isnull(total_floors):
            tf_str = str(int(total_floors))
            if num_str.endswith(tf_str):
                floor_part = num_str[:-len(tf_str)]
                if floor_part.isdigit():
                    floor_val = int(floor_part)
                    if 0 < floor_val <= 100:
                        return floor_val
        return np.nan


    dataset['floor_clean'] = dataset.apply(
        lambda row: extract_floor_strict(row['floor'], row['total_floors']),
        axis=1
    )

# פונקציה לחילוץ מספר קומה מהתיאור
    def extract_floor_from_description(desc):
        if pd.isna(desc):
            return np.nan
        desc = desc.lower()

        # קומה מספרית רגילה: "קומה 3", "בקומה 10", "קומה: 5"
        match = re.search(r'קומ[ה|ת] (?:מס\' )?(\d{1,2})', desc)
        if match:
            return int(match.group(1))

        # קומה מעל משהו – לדוגמה "מעל קומה 20"
        match = re.search(r'מעל.*?(\d{1,2})', desc)
        if match:
            return int(match.group(1)) + 1

        # קומת קרקע
        if 'קומת קרקע' in desc or 'קרקע' in desc:
            return 0

        return np.nan

    # שמירת עותק של העמודה לפני השינויים
    original_floor = dataset['floor_clean'].copy()

    # סינון שורות עם ערך חסר בקומה
    missing_floor_mask = dataset['floor_clean'].isna()
    dataset.loc[missing_floor_mask, 'floor_clean'] = dataset.loc[missing_floor_mask, 'description'].apply(extract_floor_from_description)

    
    dataset.dropna(subset=['floor_clean'], inplace=True) #1


    def fix_merged_floor(row):
        floor = row['floor_clean']
        total = row['total_floors']
        if pd.notna(floor) and pd.notna(total):
            if floor > total:
                digits = [int(d) for d in str(int(floor)) if d.isdigit()]
                if int(total) in digits:
                    digits.remove(int(total))
                    if digits:
                        new_floor = digits[0]
                        if new_floor <= total:
                            return new_floor
        return floor

    dataset['floor_clean'] = dataset.apply(fix_merged_floor, axis=1)

###############################################################################################################
    mask = (
        (dataset['floor_clean'] > dataset['total_floors']) &
        (dataset['floor_clean'].notna()) &
        (dataset['total_floors'].notna())
    )
    dataset.loc[mask, 'total_floors'] = np.nan

    def extract_total_floors_from_floor(val):
        if pd.isna(val):
            return np.nan
        val = str(val).strip()
        if val == 'קרקע מתוך קרקע':
            return 1
        if 'קרקע מתוך' in val:
            match = re.search(r'קרקע מתוך (\d+)', val)
            if match:
                return int(match.group(1))
            else:
                return np.nan
        match = re.findall(r'\d+', val)
        if len(match) >= 2:
            return int(match[-1])
        return np.nan

    mask_missing = dataset['total_floors'].isna()
    dataset.loc[mask_missing, 'total_floors'] = dataset.loc[mask_missing, 'floor'].apply(extract_total_floors_from_floor)
    dataset['total_floors'] = dataset['total_floors'].astype(float)

    # ✅ שלב 1: יצירת קטגוריות זמניות של מיסי בניין
    dataset['_building_tax_bin'] = pd.qcut(dataset['building_tax'], q=4, duplicates='drop')
    
    # ✅ שלב 2: חישוב חציון total_floors לפי elevator + has_parking + tax_bin
    group_median = dataset.groupby(['elevator', 'has_parking', '_building_tax_bin'])['total_floors'].median()
    
    # ✅ שלב 3: פונקציית מילוי חכמה שמתחשבת גם בקומה בפועל
    def smart_fill(row):
        if pd.notna(row['total_floors']):
            return row['total_floors']
    
        try:
            group_value = group_median.loc[row['elevator'], row['has_parking'], row['_building_tax_bin']]
        except:
            group_value = dataset['total_floors'].median()
    
        if pd.notna(row.get('floor_clean')):
            return max(group_value, row['floor_clean'])
        else:
            return group_value
    
    # ✅ שלב 4: הפעלת הפונקציה על כל השורות
    dataset['total_floors'] = dataset.apply(smart_fill, axis=1)
    
    # ✅ שלב 5: הסרת העמודה הזמנית
    dataset.drop('_building_tax_bin', axis=1, inplace=True)

  
    
########################################################################################################################################
    # עיבוד garden_area
    def extract_garden_area(desc):
        if pd.isna(desc):
            return np.nan
        desc = desc.replace(',', '')
        match = re.search(r'גינה[^0-9]{0,10}(\d{1,4}) ?מ"ר', desc)
        if match:
            try:
                area = int(match.group(1))
                if 1 <= area <= 300:
                    return area
            except:
                return np.nan
        return np.nan

    dataset['garden_area_filled'] = dataset['garden_area']
    missing_mask = dataset['garden_area'].isna()
    dataset.loc[missing_mask, 'garden_area_filled'] = dataset.loc[missing_mask, 'description'].apply(extract_garden_area)

    mask = (
        dataset['garden_area_filled'].isna() &
        (dataset['floor_clean'] > 3) &
        (dataset['has_balcony'] == 0) &
        (dataset['property_type_clean'].isin(['דירה', 'דופלקס']))
    )
    dataset.loc[mask, 'garden_area'] = 0
    dataset['garden_area_filled'] = dataset['garden_area_filled'].fillna(0)
########################################################################################################################
    def categorize_garden_area(x):
        if pd.isna(x):
            return 'לא ידוע'
        elif x == 0:
            return 'אין גינה'
        elif x <= 20:
            return 'קטנה'
        elif x <= 50:
            return 'בינונית'
        else:
            return 'גדולה'

    dataset['garden_size_category'] = dataset['garden_area'].apply(categorize_garden_area)
#########################################################################################################################
    # עיבוד room_num
 # ✅ שלב מקדים: החלפת אפסים ב-NaN
    dataset['room_num'] = dataset['room_num'].replace(0, np.nan)
    
    # ✅ פונקציית חילוץ מחדרים
    def extract_room_num_from_description(description):
        if pd.isnull(description):
            return np.nan
    
        description = str(description).strip()
    
        hebrew_numbers = {
            'חדר אחד': 1,
            'חדר וחצי': 1.5,
            'חדרים וחצי': 1.5,
            'שניים': 2,
            'שני': 2,
            'שתי': 2,
            'שלושה': 3,
            'שלוש': 3,
            'ארבעה': 4,
            'ארבע': 4,
            'חמישה': 5,
            'חמש': 5,
            'שישה': 6,
            'שש': 6,
            'שבעה': 7,
            'שבע': 7,
            'שמונה': 8,
            'תשעה': 9,
            'תשע': 9,
            'עשרה': 10
        }

        # 1. תבניות כמו "3 חדרים", "2.5 חדרים"
        match = re.search(r'(?:דירת|כוללת)?\s*(\d(?:[.,]\d)?)\s*חדרים?', description)
        if match:
            try:
                return float(match.group(1).replace(',', '.'))
            except:
                return np.nan
    
        # 2. מספר לפני "חדר"
        match = re.search(r'(\d(?:[.,]\d)?)\s*חדר', description)
        if match:
            try:
                return float(match.group(1).replace(',', '.'))
            except:
                return np.nan
    
        # 3. מיפוי של מילים
        for word_form, number in hebrew_numbers.items():
            if re.search(rf'{word_form}\s*חדר', description):
                return number
    
        # 4. fallback – רק "חדר"
        if re.search(r'\bחדר\b', description):
            return 1
    
        return np.nan
    
    # ✅ יצירת עמודה חדשה
    dataset['room_num_from_desc'] = dataset['description'].apply(extract_room_num_from_description)
    
    # ✅ מילוי לפי תיאור
    condition = (
        dataset['room_num'].isna() &
        dataset['room_num_from_desc'].notna() &
        (dataset['room_num_from_desc'] != 0)
    )
    dataset.loc[condition, 'room_num'] = dataset.loc[condition, 'room_num_from_desc']
    
    # ✅ אינדקציה על שורות שיוסרו
    before_drop = len(dataset)
    dataset.dropna(subset=['room_num'], inplace=True)
    after_drop = len(dataset)
    print(f"❌ הוסרו {before_drop - after_drop} שורות ללא ערך ב-room_num לאחר ניסיון המילוי מתיאור.")

################################################################################################################
# עיבוד area
    Q1 = dataset['area'].quantile(0.25)
    Q3 = dataset['area'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    mask_outliers = (dataset['area'] < 15) | (dataset['area'] > upper_bound)
    dataset.loc[mask_outliers, 'area'] = np.nan

    median_area_by_room = dataset.groupby('room_num')['area'].median()
    dataset['area'] = dataset.apply(
        lambda row: median_area_by_room[row['room_num']] if pd.isna(row['area']) else row['area'],
        axis=1
    )
#################################################################################################################
    # עיבוד building_tax
    Q1 = dataset['building_tax'].quantile(0.25)
    Q3 = dataset['building_tax'].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    lower_bound = 0

    high_outliers_mask = dataset['building_tax'] > 1500
    dataset.loc[high_outliers_mask, 'building_tax'] = np.nan

    low_outliers_mask = (dataset['building_tax'] < 50) & (dataset['building_tax'] != 0)
    dataset.loc[low_outliers_mask, 'building_tax'] = np.nan

    mask = dataset['property_type_clean'] == 'פרטי'
    dataset.loc[mask, 'building_tax'] = 0

    def categorize_building_tax(value):
        if pd.isna(value):
            return "לא יודע"
        elif 0 <= value < 500:
            return "זול"
        elif 500 <= value < 1000:
            return "ממוצע"
        elif 1000 <= value <= 1500:
            return "גבוה"
        else:
            return "לא ידוע"

    dataset["building_tax_category"] = dataset["building_tax"].apply(categorize_building_tax)
    dataset["building_tax_category"] = dataset["building_tax_category"].astype("category")

    def extract_building_tax(desc):
        if pd.isna(desc):
            return np.nan
        desc = desc.replace(',', '')
        patterns = [
            r'מיס[י|י] [ה]?בנ[יי]ן[^0-9]{0,10}(\\d{2,5})',
            r'ועד[^0-9]{0,10}(\\d{2,5})',
            r'עד בנ[יי]ן[^0-9]{0,10}(\\d{2,5})',
        ]
        for pattern in patterns:
            match = re.search(pattern, desc)
            if match:
                try:
                    value = int(match.group(1))
                    if 0 < value < 3000:
                        return value
                except:
                    return np.nan
        return np.nan

    missing_mask = dataset['building_tax'].isna()
    dataset.loc[missing_mask, 'building_tax'] = dataset.loc[missing_mask, 'description'].apply(extract_building_tax)

    dataset['total_floors_round'] = dataset['total_floors'].round()
    group1 = dataset.groupby(['has_parking', 'total_floors_round', 'has_safe_room'])['building_tax'].median()
    group2 = dataset.groupby(['property_type_clean', 'neighborhood'])['building_tax'].median()

    def fill_building_tax(row):
        if pd.notna(row['building_tax']):
            return row['building_tax']
        try:
            val = group1.loc[row['has_parking'], row['total_floors_round'], row['has_safe_room']]
            if pd.notna(val):
                return val
        except:
            pass
        try:
            val = group2.loc[row['property_type_clean'], row['neighborhood']]
            if pd.notna(val):
                return val
        except:
            pass
        return dataset['building_tax'].median()

    dataset['building_tax'] = dataset.apply(fill_building_tax, axis=1)
    dataset.drop('total_floors_round', axis=1, inplace=True)
################################################################################################################################
    # עיבוד monthly_arnona
    dataset.loc[dataset['monthly_arnona'] == 0, 'monthly_arnona'] = np.nan
    dataset.loc[dataset['monthly_arnona'] < 100, 'monthly_arnona'] = np.nan
    dataset.loc[dataset['monthly_arnona'] > 5000, 'monthly_arnona'] = np.nan

    def fill_missing_arnona(row):
        if pd.notna(row['monthly_arnona']):
            return row['monthly_arnona']

        area_min = row['area'] - 5
        area_max = row['area'] + 5
        rooms_min = row['room_num'] - 1
        rooms_max = row['room_num'] + 1

        similar = dataset[
            (dataset['neighborhood'] == row['neighborhood']) &
            (dataset['area'] >= area_min) & (dataset['area'] <= area_max) &
            (dataset['room_num'] >= rooms_min) & (dataset['room_num'] <= rooms_max) &
            (dataset['monthly_arnona'].notna())
        ]

        if not similar.empty:
            return similar['monthly_arnona'].mean()
        else:
            return np.nan

    dataset['monthly_arnona'] = dataset.apply(fill_missing_arnona, axis=1)

    def fill_arnona_by_address_or_neighborhood(row):
        if pd.notna(row['monthly_arnona']):
            return row['monthly_arnona']

        if pd.notna(row['street']):
            street_mask = (dataset['street'] == row['street']) & dataset['monthly_arnona'].notna()
            if street_mask.sum() >= 3:
                return dataset.loc[street_mask, 'monthly_arnona'].median()

        if pd.notna(row['neighborhood']):
            neigh_mask = (dataset['neighborhood'] == row['neighborhood']) & dataset['monthly_arnona'].notna()
            if neigh_mask.sum() >= 3:
                return dataset.loc[neigh_mask, 'monthly_arnona'].median()

        return np.nan

    dataset['monthly_arnona'] = dataset.apply(fill_arnona_by_address_or_neighborhood, axis=1)

    def fill_arnona_by_distance_similarity(row, median_distance_by_neigh, median_arnona_by_neigh, global_median):
        if pd.notna(row['monthly_arnona']):
            return row['monthly_arnona']

        if pd.isna(row['distance_from_center']):
            return global_median

        diffs = (median_distance_by_neigh - row['distance_from_center']).abs()
        closest_neigh = diffs.idxmin()
        value = median_arnona_by_neigh.get(closest_neigh, np.nan)
        if pd.isna(value):
            return global_median
        return value

    median_distance_by_neigh = dataset.groupby('neighborhood')['distance_from_center'].median()
    median_arnona_by_neigh = dataset.groupby('neighborhood')['monthly_arnona'].median()
    global_median_arnona = dataset['monthly_arnona'].median()

    dataset['monthly_arnona'] = dataset.apply(
        lambda row: fill_arnona_by_distance_similarity(
            row,
            median_distance_by_neigh,
            median_arnona_by_neigh,
            global_median_arnona
        ),
        axis=1
    )
#########################################################################################################################
# עיבוד entering_category
    def map_days_to_category(val):
        if pd.isna(val):
            return 'unknown'
        elif val == 0:
            return 'immediate'
        elif val == -1:
            return 'flexible'
        else:
            return 'in_future'

    dataset['entering_category'] = dataset['days_to_enter'].apply(map_days_to_category)
##########################################################################################################################
    # עיבוד num_of_payments
    dataset['num_of_payments'] = dataset['num_of_payments'].replace(0, np.nan)
    dataset['num_of_payments'] = dataset['num_of_payments'].fillna(dataset['num_of_payments'].mode()[0])
    
    # num_of_images
    dataset['num_of_images'] = dataset['num_of_images'].fillna(0)

    #עיבוד distance_from_center
    def fill_missing_distances_via_api(dataset, api_key, center_address="כיכר דיזנגוף, תל אביב"):
        def format_address(addr):
            if not isinstance(addr, str) or not addr.strip():
                return None
            if "תל אביב" not in addr:
                addr += ", תל אביב"
            return addr

#         def get_distance(origin, destination, api_key):
#             url = f"https://routes.googleapis.com/directions/v2:computeRoutes"
#             headers = {
#                 'Content-Type': 'application/json',
#                 'X-Goog-Api-Key': api_key,
#                 'X-Goog-FieldMask': 'routes.distanceMeters'
#             }
#             body = {
#                 "origin": {"address": origin},
#                 "destination": {"address": destination},
#                 "travelMode": "DRIVE",
#                 "routingPreference": "TRAFFIC_AWARE"
#             }

#             response = requests.post(url, headers=headers, json=body)
#             if response.status_code == 200:
#                 data = response.json()
#                 if 'routes' in data and data['routes']:
#                     return float(data['routes'][0]['distanceMeters'])
#             return None

#         missing = dataset[dataset['distance_from_center'].isna()].copy()

#         for i, row in missing.iterrows():
#             address = format_address(row.get('address')) or format_address(row.get('street'))
#             if not address:
#                 continue

#             distance = get_distance(address, center_address, api_key)
#             time.sleep(0.2)

#             if distance is not None:
#                 dataset.at[i, 'distance_from_center'] = distance

#         return dataset
#     api_key =getpass.getpass()
#     dataset = fill_missing_distances_via_api(dataset, api_key=api_key)    
#     dataset.dropna(subset=['distance_from_center'], inplace=True)
      
    # Fill missing values in 'distance_from_center' with the global median
    median_distance = dataset['distance_from_center'].median()
    dataset['distance_from_center'].fillna(median_distance, inplace=True)

    
    
    
    
    return dataset  
    
    
    
    
    

In [6]:
my_data_test=prepare_data(my_data_test)


❌ הוסרו 2 שורות ללא price
❌ הוסרו 53 שורות עם מחיר מחוץ לטווח 1500–18500
❌ הוסרו 2 שורות שבהן גם neighborhood וגם address חסרים
❌ הוסרו 0 שורות עם 4 ערכים או פחות (לא 0 ולא NaN)
❌ הוסרו 1 שורות עם התייחסות ל'קורות חיים'
❌ הוסרו 11 שורות שבהן סוג הנכס לא זוהה (לא ידוע)

📊 סך הכול נשארו 719 שורות מתוך 731 המקוריות.


C:\Users\ngr25\AppData\Local\Temp\ipykernel_12348\488166565.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.replace({'': np.nan, ' ': np.nan}, inplace=True)


❌ הוסרו 7 שורות ללא ערך ב-room_num לאחר ניסיון המילוי מתיאור.


In [7]:
luxury_types = ['גג/פנטהאוז', 'דופלקס']
my_data_test['is_luxury_type'] = my_data_test['property_type'].isin(luxury_types).astype(int)

In [8]:
features=['property_type_clean', 'neighborhood', 'street', 'room_num', 'floor_clean', 'area',
       'garden_area_filled', 'garden_size_category','entering_category', 'num_of_payments', 'monthly_arnona',
       'building_tax', 'total_floors', 'has_parking',
       'has_storage', 'elevator', 'ac', 'handicap', 'has_bars',
       'has_safe_room', 'has_balcony', 'is_furnished', 'is_renovated',
       'num_of_images', 'distance_from_center', 'price','building_tax_category' ]
my_data_test=my_data_test[features]

In [9]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ---------- Define selected features ----------
features = ['property_type_clean', 'neighborhood', 'street', 'room_num', 'floor_clean', 'area',
            'garden_area_filled', 'garden_size_category', 'entering_category', 'num_of_payments',
            'monthly_arnona', 'building_tax', 'total_floors', 'has_parking', 'has_storage', 'elevator',
            'ac', 'handicap', 'has_bars', 'has_safe_room', 'has_balcony', 'is_furnished', 'is_renovated',
            'num_of_images', 'distance_from_center', 'price', 'building_tax_category']

# ---------- Select data ----------
df = my_data_test[features].copy()

# ---------- Split into X (features) and y (target) ----------
X = df.drop("price", axis=1)
y = df["price"]

# ---------- Identify column types ----------
categorical_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
numeric_cols = X.select_dtypes(include=["int64", "float64", "bool"]).columns.tolist()

# ---------- Define transformations for categorical columns ----------
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # fixed here
])

# ---------- Define transformations for numeric columns ----------
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", MinMaxScaler())
])

# ---------- Combine all transformations ----------
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])

# ---------- Create pipeline with preprocessing and Random Forest model ----------
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42))
])

# ---------- Split data into train/test sets ----------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ---------- Train the model ----------
pipeline.fit(X_train, y_train)

# ---------- Save the full pipeline as trained_model.pkl
with open("trained_model.pkl", "wb") as f:
    pickle.dump(pipeline, f)

# ---------- Make predictions ----------
predictions = pipeline.predict(X_test)

# ---------- Evaluate model performance ----------
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print(f"📊 MAE: {mae:.2f} ₪")
print(f"📉 RMSE: {rmse:.2f} ₪")
print(f"📈 R²: {r2:.4f}")

# ---------- Feature importance ----------
feature_names = numeric_cols + list(
    pipeline.named_steps['preprocessor']
    .named_transformers_['cat']
    .named_steps['encoder']
    .get_feature_names_out(categorical_cols)
)

importances = pipeline.named_steps["regressor"].feature_importances_

importance_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values(by="importance", ascending=False)

top5 = importance_df.head(5)
print("\n🌟 Top 5 Most Important Features:")
print(top5.to_string(index=False))


📊 MAE: 1546.85 ₪
📉 RMSE: 2184.83 ₪
📈 R²: 0.6331

🌟 Top 5 Most Important Features:
             feature  importance
                area    0.446767
        building_tax    0.107908
      monthly_arnona    0.055803
            room_num    0.047291
distance_from_center    0.035921


## Saving the model

In [10]:
# save the model:

import pickle
pickle.dump(pipeline, open("trained_model.pkl", "wb"))



## loading a saved model


In [11]:
pipeline = pickle.load(open("trained_model.pkl","rb"))
